# Garbage collecion: сборка мусора (Модуль gc)

Модуль __gc__ предоставляет базовый механизм управления памятью в Python — автоматическую сборку мусора. Он включает функции, позволяющие принудительно вызывать сборщик мусора и исследовать известные системе объекты, которые либо намечены для удаления, либо связаны циклическими ссылками и не могут быть освобождены.

## Отслеживание ссылок

Анализируя существующие между объектами входящие и исходящие ссылки, модуль __gc__ способен выявлять наличие циклических ссылок в сложных структурах данных. Если речь идет о циклических ссылках в известной структуре, то ее свойства могут быть исследованы c помощью пользовательского кода. Если же циклические ссылки появляются в неизвестном коде, то для создания обобщенных средств отладки можно использовать функции:

Например, функция __get_referents()__ отображает объекты, на которые ссылаются входные аргументы.

In [1]:
import gc
import pprint

class Graph:
    
    def __init__(self, name):
        self.name = name
        self.next = None
        
    def set_next(self, next):
        print ('Linking nodes {}.next={}' .format(self, next))
        self.next = next
        
    def __repr__(self):
        return '{}({})' .format(
            self.__class__.__name__, self.name) 
    
#Cоздать циклический граф
one = Graph('one')
two = Graph('two')
three = Graph('three')

one.set_next(two)
two.set_next(three)
three.set_next(one)

print()
print('three refers to:')
for r in gc.get_referents(three):
    pprint.pprint(r)

Linking nodes Graph(one).next=Graph(two)
Linking nodes Graph(two).next=Graph(three)
Linking nodes Graph(three).next=Graph(one)

three refers to:
{'name': 'three', 'next': Graph(one)}
<class '__main__.Graph'>


В данном случае экземпляр three класса Graph содержит ссылки на свой словарь и свой класс.

## Принудительная сборка мусора

Несмотря на то, что сборщик мусора выполняется автоматически по мере выполнения программы интерпретатором, его можно запускать в определенные моменты времени, если необходимо освободить память от большого количества ненужных объектов или если выполняется лишь небольшой объем полезной работы и запуск сборщика не повлияет на производительность программы. 
Для этого следует использовать функцию __collect()__

In [3]:
import gc
import pprint

class Graph:
    
    def __init__(self, name):
        self.name = name
        self.next = None
        
    def set_next(self, next):
        print ('Linking nodes {}.next={}' .format(self, next))
        self.next = next
        
    def __repr__(self):
        return '{}({})' .format(
            self.__class__.__name__, self.name) 
    
#Cоздать циклический граф
one = Graph('one')
two = Graph('two')
three = Graph('three')

one.set_next(two)
two.set_next(three)
three.set_next(one)

# Удалить ссылки на узлы графа в пространстве имен данного модуля
one = two = three = None

# Продемонстрировать эффект сборки мусора
for i in range(2):
    print('\nCollecting {} ...'.format(i))
    n = gc.collect()
    print('Unreachable objects:', n)
    print('Remaining Garbage:', end=' ')
    pprint.pprint(gc.garbage)

Linking nodes Graph(one).next=Graph(two)
Linking nodes Graph(two).next=Graph(three)
Linking nodes Graph(three).next=Graph(one)

Unreachable objects: 36
Remaining Garbage: []

Unreachable objects: 0
Remaining Garbage: []


В этом примере цикл удаляется сразу же, как только выполняется сборка мусора, поскольку на узлы Graph не ссылаются никакие другие объекты, кроме них самих. Функция __collect()__ возвращает количество “недостижимых” объектов, которые ей удалось обнаружить. В данном случае это значение представляет три объекта c их словарями атрибутов экземпляров.

## Пороги и поколения сборки мусора

Сборщик мусора поддерживает три списка объектов — по одному на каждое поколение сборки, отслеживаемое им. В процессе просмотра объектов в каждом поколении они либо удаляются, либо переводятся в следующее поколение, пока окончательно не достигнут состояния, в котором они постоянно находятся.
Выполняемые сборщиком рутинные операции можно настраивать так, чтобы они осуществлялись c различной частотой на основании разницы между количествами размещаемых и удаляемых объектов от запуска к запуску. Если количество размещений объектов минус количество удалений превышает установленный для
данного поколения порог, запускается сборщик мусора. Для получения текущих пороговых значений следует использовать функцию __get_threshold()__

In [2]:
import gc

print(gc.get_threshold())

(700, 10, 10)


Возвращаемое значение представляет собой кортеж из пороговых значений для каждого поколения.

Для изменения пороговых значений следует использовать функцию __set_threshold()__. В следующем примере программа устанавливает для поколения 0 пороговое значение, заданное аргументом командной строки, а затем размещает в памяти последовательность объектов.

In [4]:
import gc
import pprint
import sys

try:
    threshold = int(sys.argv[1])
except (IndexError, ValueError, TypeError):
    print('Missing or invalid threshold, using default')
    threshold = 2

          
class MyObj:
          
    def __init__ (self, name):
        self.name = name
        print('Created', self.name)
          
gc.set_debug(gc.DEBUG_STATS)
gc.set_threshold(threshold, 1, 1)
print('Thresholds:', gc.get_threshold())
          
print('Clear the collector by forcing a run')
gc.collect()
print()
          
print('Creating objects')
objs = []
for i in range(10):
    objs.append(MyObj(i))
print('Exiting')
          
# Отключить отладку
gc.set_debug(0)

Missing or invalid threshold, using default
Thresholds: (2, 1, 1)
Clear the collector by forcing a run

Creating objects


gc: collecting generation 0...
gc: objects in each generation: 26 0 50936
gc: objects in permanent generation: 0
gc: done, 0 unreachable, 0 uncollectable, 0.0000s elapsed
gc: collecting generation 1...
gc: objects in each generation: 11 28 50936
gc: objects in permanent generation: 0
gc: done, 0 unreachable, 0 uncollectable, 0.0000s elapsed
gc: collecting generation 0...
gc: objects in each generation: 34 0 50977
gc: objects in permanent generation: 0
gc: done, 0 unreachable, 0 uncollectable, 0.0000s elapsed
gc: collecting generation 0...
gc: objects in each generation: 17 31 50957
gc: objects in permanent generation: 0
gc: done, 0 unreachable, 0 uncollectable, 0.0000s elapsed
gc: collecting generation 1...
gc: objects in each generation: 18 47 50950
gc: objects in permanent generation: 0
gc: done, 0 unreachable, 0 uncollectable, 0.0000s elapsed
gc: collecting generation 0...
gc: objects in each generation: 9 0 51019
gc: objects in permanent generation: 0
gc: done, 0 unreachable, 0 unc

Created 0
Created 1
Created 2
Created 3
Created 4
Created 5
Created 6
Created 7
Created 8
Created 9
Exiting


gc: done, 0 unreachable, 0 uncollectable, 0.0000s elapsed
gc: collecting generation 0...
gc: objects in each generation: 9 0 50935
gc: objects in permanent generation: 0
gc: done, 0 unreachable, 0 uncollectable, 0.0000s elapsed
gc: collecting generation 0...
gc: objects in each generation: 9 28 50918
gc: objects in permanent generation: 0
gc: done, 0 unreachable, 0 uncollectable, 0.0000s elapsed
gc: collecting generation 1...
gc: objects in each generation: 9 41 50918
gc: objects in permanent generation: 0
gc: done, 0 unreachable, 0 uncollectable, 0.0000s elapsed
gc: collecting generation 0...
gc: objects in each generation: 9 0 50964
gc: objects in permanent generation: 0
gc: done, 0 unreachable, 0 uncollectable, 0.0000s elapsed
gc: collecting generation 0...
gc: objects in each generation: 9 26 50963
gc: objects in permanent generation: 0
gc: done, 0 unreachable, 0 uncollectable, 0.0000s elapsed
gc: collecting generation 1...
gc: objects in each generation: 9 39 50963
gc: objects in 

Изменение пороговых значений влияет на то, в какие моменты времени будет выполняться сборка мусора. При меньших пороговых значениях сборка мусора выполняется чаще.

## Отладка

Обнаружение причин утечки памяти может оказаться трудной задачей. Модуль gc предоставляет несколько отладочных опций, которые упрощают решение этой задачи, позволяя увидеть детали работы внутренних механизмов кода. Указанные опции представляют собой битовые флаги, которые можно комбинировать и передавать функции __set_debug()__ для управления поведением сборщика мусора во время выполнения программы. Отладочная информация выводится в поток __sys.stderr__.


Флаг __DEBUG_STATS__ включает вывод статистики, вынуждая механизм сборщика мусора выводить в процессе своего выполнения информацию о том, сколько объектов отмечено для помещения в каждое поколение и сколько времени требуется для очистки памяти.

In [9]:
import gc

gc.set_debug(gc.DEBUG_STATS)

gc.collect()
print('Exiting')

Exiting


gc: collecting generation 0...
gc: objects in each generation: 26 0 50747
gc: objects in permanent generation: 0
gc: done, 0 unreachable, 0 uncollectable, 0.0000s elapsed
gc: collecting generation 0...
gc: objects in each generation: 11 28 50747
gc: objects in permanent generation: 0
gc: done, 0 unreachable, 0 uncollectable, 0.0000s elapsed
gc: collecting generation 2...
gc: objects in each generation: 13 27 50747
gc: objects in permanent generation: 0
gc: done, 26 unreachable, 0 uncollectable, 0.0160s elapsed
gc: collecting generation 0...
gc: objects in each generation: 24 0 50754
gc: objects in permanent generation: 0
gc: done, 0 unreachable, 0 uncollectable, 0.0000s elapsed
gc: collecting generation 0...
gc: objects in each generation: 16 23 50754
gc: objects in permanent generation: 0
gc: done, 0 unreachable, 0 uncollectable, 0.0000s elapsed
gc: collecting generation 1...
gc: objects in each generation: 25 43 50753
gc: objects in permanent generation: 0
gc: done, 0 unreachable, 0 

Флаги __DEBUG_COLLECTABLE__ и __DEBUG_UNCOLLECTABLE__ вынуждают сборщик мусора выводить информацию о том, может или не может быть удален проверяемый объект. Если информации об объектах, которые не могут быть удалены, еще недостаточно для того, чтобы определить, где именно удерживаются данные, следует установить флаг __DEBUG_SAVEALL__, вынуждающий модуль gc сохранить все обнаруженные объекты, на которые отсутствуют ссылки, в списке __garbage__.

In [5]:
import gc
flags = gc.DEBUG_LEAK

gc.set_debug(flags)

class Graph:
    
    def __init__(self, name):
        self.name = name
        self.next = None
        
    def set_next(self, next):
        self.next = next
        
    def __repr__(self) :
        return '{}({})'.format(
            self.__class__.__name__, self.name)

class CleanupGraph(Graph):
    
    def __del__(self) :
        print('{}.__del__()'.format(self))
              
# Создать циклический граф
one = Graph('one')
two = Graph('two')
one.set_next(two)
two.set_next(one)
            
# Создать самостоятельный узел
three = CleanupGraph('three')
                     
# Создать циклический граф c помощью финализатора
four = CleanupGraph('four')
five = CleanupGraph('five')
four.set_next(five)
five.set_next(four)
                    
# Удалить ссылки на узлы графа в пространстве имен этого модуля
one = two = three = four = five = None
                    
# Принудительный запуск сборщика мусора
print('Collecting')
gc.collect ()
print('Done')
      
# Вывести информацию об оставшихся объектах
for o in gc.garbage:
    if isinstance(o, Graph):
        print('Retained: {} Ox{:x}'.format(o, id(o)))
      
    
# Сбросить флаги отладки перед выходом    
gc.set_debug(0)

CleanupGraph(three).__del__()
Collecting
CleanupGraph(four).__del__()
CleanupGraph(five).__del__()
Done
Retained: Graph(one) Ox1f0c7483f70
Retained: Graph(two) Ox1f0c7483ee0
Retained: CleanupGraph(four) Ox1f0c7483d90
Retained: CleanupGraph(five) Ox1f0c7483640


gc: collectable <Graph 0x000001F0C7456D00>
gc: collectable <Graph 0x000001F0C7456D60>
gc: collectable <Graph 0x000001F0C7456E50>
gc: collectable <dict 0x000001F0C7477C80>
gc: collectable <dict 0x000001F0C7477E80>
gc: collectable <dict 0x000001F0C7477DC0>
gc: collectable <frame 0x000001F0C59B2120>
gc: collectable <list 0x000001F0C7446900>
gc: collectable <dict 0x000001F0C7477200>
gc: collectable <dict 0x000001F0C747C240>
gc: collectable <list 0x000001F0C743FE00>
gc: collectable <type 0x000001F0C69D08F0>
gc: collectable <dict 0x000001F0C7446280>
gc: collectable <tuple 0x000001F0C73EC440>
gc: collectable <function 0x000001F0C748B040>
gc: collectable <getset_descriptor 0x000001F0C747CE40>
gc: collectable <getset_descriptor 0x000001F0C747CF40>
gc: collectable <ValueError 0x000001F0C5C50EA0>
gc: collectable <traceback 0x000001F0C73EE5C0>
gc: collectable <frame 0x000001F0C73A5C10>
gc: collectable <ValueError 0x000001F0C5C50F40>
gc: collectable <frame 0x000001F0C5ADE170>
gc: collectable <frame

Для простоты определен флаг __DEBUG_LEAK__, представляющий комбинацию
всех остальных опций. Он автоматически включает флаг __DEBUG_
SAVEALL__, в связи c чем будут сохранены даже те объекты, на которые отсутствуют
ссылки и которые в обычных условиях были бы затребованы сборщиком мусора
и в конечном счете удалены.